## Foundations of Data Science - 2020 - Exercise Sheet 8

### Exercise 8.1 (The greedy algorithm works). We would like to check that the greedy algorithm works in a further sense:
### compare the definition to the singular value decomposition, ie. numerically the result of the numpy command numpy.linalg.svd()

In [1]:
import numpy as np

#### (ii) Write a Python routine first(), that for any matrix A returns the first singular vector. You may use numpy.linalg.svd() for this.

As on previous slides, we use the conjugated eigen vector matrix $V^{*}$. Since conjugating is transposing on real matrices, we take the first row of $V^{*}$ instead of the first column to get the first singular vector.

In [2]:
def first(A):
    _, _, vT = np.linalg.svd(A, full_matrices=False)
    return vT[0]

#### (iii) Randomly generate a matrix for at least dimension d=4 and sample size n=8 (to get an $n×d$-matrix). Apply this routine to the matrix to obtain the first singular vector and apply (i) to also obtain the second.

There are two possible obstacles here. First, we need to tell numpy to use the outer product of the eigenvectors. This can be done with giving the correct shapes to vanilla np.dot, but numpy offers a function to do this directly. Second, we want to subtract the outer product from the identity. If we provide just the scalar $1$, numpy assumes we are looking for component wise subtraction.

In [3]:
np.random.seed(42)
A = np.random.randint(0, 10, (8,4))

firstSingularVector = first(A)

A_1trunc = A @ (np.identity(firstSingularVector.shape[0]) - np.outer(firstSingularVector, firstSingularVector))
secondSingularVector = first(A_1trunc)

print("First singular vector by using first():", firstSingularVector)
print("Second singular vector by using (i):   ", secondSingularVector)

First singular vector by using first(): [-0.5918612  -0.33421419 -0.61367994 -0.40174386]
Second singular vector by using (i):    [ 0.05163298 -0.78954908  0.55219367 -0.26273264]


#### (iv) Compare numerically the second singular vector to the vector $v_{1}$ from the SVD given by numpy.linalg.svd().

Again, we select the first *row* of vT, since it is transposed.

In [4]:
_, _, vT = np.linalg.svd(A)
print("Second singular vector by using (i):           ", secondSingularVector)
print("Second singular vector by using numpy directly:", vT[1], "\n")

Second singular vector by using (i):            [ 0.05163298 -0.78954908  0.55219367 -0.26273264]
Second singular vector by using numpy directly: [ 0.05163298 -0.78954908  0.55219367 -0.26273264] 



#### (v) With a method similar to (i), compute also the third singular vector and compare it to the SVD.

In [5]:
A_2trunc = A_1trunc @ (np.identity(secondSingularVector.shape[0]) - np.outer(secondSingularVector, secondSingularVector))
thirdSingularVector = first(A_2trunc)
print("Third singular vector by using (i):            ", thirdSingularVector)
print("Third singular vector by using numpy directly: ", vT[2])

Third singular vector by using (i):             [ 0.47860695 -0.49892388 -0.51895629  0.5026875 ]
Third singular vector by using numpy directly:  [-0.47860695  0.49892388  0.51895629 -0.5026875 ]


### Power Method to calculate the first singular vector

In [6]:
def PM0(A, l):
    np.random.seed(42)
    v = np.random.normal(loc=0, scale=1, size=A.shape[1]).astype(np.double)
    # numba
    for it in range(l):
        v = np.dot(A, v) # shape of v from d to n
        v = np.dot(A.T, v) # shape of v from
        v = v / np.abs(v).max()

    v0 = v / np.linalg.norm(v)
    
    return v0
        
        

### Some examples and calculations

In [7]:
np.random.seed(42)
A = np.random.randint(0, 10, (8,4)).astype(np.double)
u, s, v = np.linalg.svd(A, full_matrices=False)

print(v.T)
l = 5
print("sigma:", s)
print("error:",np.abs(PM0(A, l)) - np.abs(PM0(A, 20)))
print("ratio:", (s[1] / s[0])**(l*2))

[[-0.5918612   0.05163298 -0.47860695 -0.64650579]
 [-0.33421419 -0.78954908  0.49892388 -0.12644402]
 [-0.61367994  0.55219367  0.51895629  0.22172835]
 [-0.40174386 -0.26273264 -0.5026875   0.71894278]]
sigma: [27.79810202 10.64200793  6.60385764  3.79503037]
error: [ 4.15781593e-07 -4.08809737e-06  2.56832729e-06 -1.13487275e-06]
ratio: 6.762142785708386e-05
